In [199]:

from numpy import loadtxt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import LinearSVC, SVC
import eli5


In [200]:
def classifier(num=0, rfnest=1000, rfdep=10, SVMC=10):
    if num==0:
        return RandomForestClassifier(n_estimators=rfnest,
                                     criterion='gini',
                                     max_depth=rfdep,
                                     min_samples_split=2,
                                     min_samples_leaf=1,
                                     min_weight_fraction_leaf=0.0,
                                     max_features=None,
                                     max_leaf_nodes=None,
                                     bootstrap=True,
                                     oob_score=False,
                                     n_jobs=1,
                                     class_weight='balanced')
    elif num == 1:
        return LogisticRegression(C=10000, penalty='l2',fit_intercept=False)
    elif num == 2:
        return LinearSVC(C=SVMC,fit_intercept=False)
    elif num ==3:
        return RidgeClassifier(alpha=2, fit_intercept=False)
    elif num == 4:
        return SVC(C=10000, gamma=0.01, kernel='rbf',degree=3)

In [201]:
from IPython.core.display import display, HTML
show_html = lambda html: display(HTML(html))
show_html_expl = lambda expl, **kwargs: show_html(eli5.format_as_html(expl, include_styles=False, **kwargs))
show_html(eli5.format_html_styles())

In [202]:
data = loadtxt('CVIrf.csv', delimiter=';', skiprows=1)
fnames = ['round','cadence','vel','sLength','sTime','sLengthCV','sTimeCV','rhfx','lhfx','rhfy','lhfy','rhfz','lhfz']
ncl = 6
X = data[:, 1:-ncl]
sc = MinMaxScaler()
X= sc.fit_transform(X)

n_splits = 10

#  3 strides 3 clusters

In [203]:

yv = data[:, -ncl:]
print(yv.shape)
nlb = 0
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [204]:

skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=500, rfdep=8)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.79125


In [205]:
clf= classifier(num=0, rfnest=500, rfdep=8)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        50
        2.0       1.00      1.00      1.00        37
        3.0       1.00      1.00      1.00        66

avg / total       1.00      1.00      1.00       153

[[50  0  0]
 [ 0 37  0]
 [ 0  0 66]]
ACC= 1.0


In [206]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.4506 ± 0.1043,rhfx
0.1452 ± 0.1817,rhfz
0.0986 ± 0.2001,lhfz
0.0599 ± 0.1105,sTimeCV
0.0519 ± 0.0862,lhfx
0.0334 ± 0.0824,cadence
0.0318 ± 0.0741,sTime
0.0277 ± 0.0905,sLength
0.0269 ± 0.0625,rhfy
0.0237 ± 0.0695,sLengthCV


## SVM

In [207]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=0.1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.802797619047619


In [208]:
clf= classifier(num=2, SVMC=0.1)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.88      0.72      0.79        61
        2.0       0.43      0.73      0.54        22
        3.0       0.98      0.93      0.96        70

avg / total       0.86      0.82      0.83       153

[[44 17  0]
 [ 5 16  1]
 [ 1  4 65]]
ACC= 0.8169934640522876


In [209]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

# 5 strides 3 clusters

In [210]:
yv = data[:, -ncl:]
print(yv.shape)
nlb = 1
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [211]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=500, rfdep=10)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.7388873626373627


In [212]:
clf= classifier(num=0, rfnest=500, rfdep=10)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        59
        2.0       1.00      1.00      1.00        56
        3.0       1.00      1.00      1.00        38

avg / total       1.00      1.00      1.00       153

[[59  0  0]
 [ 0 56  0]
 [ 0  0 38]]
ACC= 1.0


In [213]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.3748 ± 0.1003,rhfx
0.1532 ± 0.2263,lhfz
0.1456 ± 0.2334,rhfz
0.0723 ± 0.1531,lhfy
0.0439 ± 0.0904,cadence
0.0385 ± 0.1140,sTimeCV
0.0364 ± 0.0914,vel
0.0339 ± 0.0907,sLength
0.0314 ± 0.0734,lhfx
0.0235 ± 0.0692,sLengthCV


## SVM

In [214]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=0.1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.7959111721611721


In [215]:
clf= classifier(num=2, SVMC=0.1)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.98      0.87      0.92        67
        2.0       0.89      0.77      0.83        65
        3.0       0.37      0.67      0.47        21

avg / total       0.86      0.80      0.82       153

[[58  0  9]
 [ 0 50 15]
 [ 1  6 14]]
ACC= 0.7973856209150327


In [216]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

# 5 strides 5 clusters

In [217]:
yv = data[:, -ncl:]
print(yv.shape)
nlb = 2
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [218]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=1000, rfdep=8)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.7855542986425339


In [219]:
clf= classifier(num=0, rfnest=1000, rfdep=8)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        30
        2.0       1.00      1.00      1.00        58
        3.0       1.00      1.00      1.00        18
        4.0       1.00      1.00      1.00        25
        5.0       1.00      1.00      1.00        22

avg / total       1.00      1.00      1.00       153

[[30  0  0  0  0]
 [ 0 58  0  0  0]
 [ 0  0 18  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  0 22]]
ACC= 1.0


In [220]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.3836 ± 0.1090,rhfx
0.1435 ± 0.2084,rhfz
0.1124 ± 0.2041,lhfz
0.0856 ± 0.1548,sTimeCV
0.0609 ± 0.1085,sTime
0.0409 ± 0.0732,rhfy
0.0334 ± 0.0621,lhfx
0.0291 ± 0.0693,sLengthCV
0.0275 ± 0.0639,vel
0.0264 ± 0.0736,cadence


## SVM

In [221]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=10)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.6994909502262443


In [222]:
clf= classifier(num=2, SVMC=10)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.80      0.73      0.76        33
        2.0       0.97      0.92      0.94        61
        3.0       0.67      0.75      0.71        16
        4.0       0.52      0.65      0.58        20
        5.0       0.82      0.78      0.80        23

avg / total       0.82      0.80      0.81       153

[[24  0  0  8  1]
 [ 0 56  4  0  1]
 [ 2  2 12  0  0]
 [ 4  0  1 13  2]
 [ 0  0  1  4 18]]
ACC= 0.803921568627451


In [223]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

# 6 strides 3 clusters

In [224]:
yv = data[:, -ncl:]
print(yv.shape)
nlb = 4
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [225]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=1000, rfdep=8)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.8069871794871795


In [226]:
clf= classifier(num=0, rfnest=1000, rfdep=8)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        49
        2.0       1.00      1.00      1.00        58
        3.0       1.00      1.00      1.00        46

avg / total       1.00      1.00      1.00       153

[[49  0  0]
 [ 0 58  0]
 [ 0  0 46]]
ACC= 1.0


In [227]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.4033 ± 0.1007,rhfx
0.1808 ± 0.2406,rhfz
0.1517 ± 0.2448,lhfz
0.0664 ± 0.1004,sTime
0.0463 ± 0.0965,sTimeCV
0.0271 ± 0.0701,lhfx
0.0248 ± 0.0782,sLength
0.0244 ± 0.0634,rhfy
0.0244 ± 0.0672,lhfy
0.0232 ± 0.0774,vel


## SVM

In [228]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=0.1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.753191391941392


In [229]:
clf= classifier(num=2, SVMC=0.1)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.47      0.74      0.58        31
        2.0       0.98      0.85      0.91        67
        3.0       0.85      0.71      0.77        55

avg / total       0.83      0.78      0.79       153

[[23  1  7]
 [10 57  0]
 [16  0 39]]
ACC= 0.7777777777777778


In [230]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

# 6 strides 4 clusters

In [231]:
yv = data[:, -ncl:]
print(yv.shape)
nlb = 5
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [232]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=1000, rfdep=8)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.7968872549019608


In [233]:
clf= classifier(num=0, rfnest=1000, rfdep=8)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        32
        2.0       1.00      1.00      1.00        41
        3.0       1.00      1.00      1.00        23
        4.0       1.00      1.00      1.00        57

avg / total       1.00      1.00      1.00       153

[[32  0  0  0]
 [ 0 41  0  0]
 [ 0  0 23  0]
 [ 0  0  0 57]]
ACC= 1.0


In [234]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.3347 ± 0.1069,rhfx
0.2305 ± 0.2339,rhfz
0.0942 ± 0.1988,lhfz
0.0905 ± 0.1619,sTimeCV
0.0623 ± 0.1129,cadence
0.0376 ± 0.0926,sLengthCV
0.0337 ± 0.0779,rhfy
0.0325 ± 0.0751,lhfx
0.0256 ± 0.0645,sLength
0.0219 ± 0.0625,sTime


## SVM

In [235]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=50)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.6762033146591971


In [236]:
clf= classifier(num=2, SVMC=50)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.69      0.73      0.71        30
        2.0       0.83      0.79      0.81        43
        3.0       0.70      0.80      0.74        20
        4.0       0.96      0.92      0.94        60

avg / total       0.84      0.83      0.83       153

[[22  6  2  0]
 [ 8 34  1  0]
 [ 1  1 16  2]
 [ 1  0  4 55]]
ACC= 0.8300653594771242


In [237]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)